In [2]:
!head -10 ml-1m/ratings.dat


1::1193::5::978300760
1::661::3::978302109
1::914::3::978301968
1::3408::4::978300275
1::2355::5::978824291
1::1197::3::978302268
1::1287::5::978302039
1::2804::5::978300719
1::594::4::978302268
1::919::4::978301368


In [12]:
import pandas as pd
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter="::", engine="python", header=None)

In [18]:
ratings.columns = ["UserID::MovieID::Rating::Timestamp".split("::")]
#ratings = ratings.iloc[:1000, :]
ratings.head()


,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [26]:
pd.read_csv?

In [19]:
ratings.isnull().sum()

UserID       0
MovieID      0
Rating       0
Timestamp    0
dtype: int64

In [48]:
import numpy as np
np.unique(ratings['Rating'])

array([1, 2, 3, 4, 5])

* Do we have any duplicate ratings, i.e., a (userid, movieid) combination that is repeated more than once?

In [21]:
user_movie_pairs = zip(ratings['UserID'].values, ratings['MovieID'].values)

In [22]:
print len(user_movie_pairs), len(set(user_movie_pairs))

1000209 1000209


In [23]:
print ratings['UserID'].unique()
set(ratings['UserID'].unique()) == set(range(1, 6041))
# User ids are range(1, 6041), i.e., are all values between 1 and 6040
# We can use a simple mapping: user_index=UserID-1

[   1    2    3 ..., 6038 6039 6040]


True

In [32]:
!head ml-1m/movies.dat

1::Toy Story (1995)::Animation|Children's|Comedy
2::Jumanji (1995)::Adventure|Children's|Fantasy
3::Grumpier Old Men (1995)::Comedy|Romance
4::Waiting to Exhale (1995)::Comedy|Drama
5::Father of the Bride Part II (1995)::Comedy
6::Heat (1995)::Action|Crime|Thriller
7::Sabrina (1995)::Comedy|Romance
8::Tom and Huck (1995)::Adventure|Children's
9::Sudden Death (1995)::Action
10::GoldenEye (1995)::Action|Adventure|Thriller


In [53]:
len(movies.MovieID) == len(movies.MovieID.unique())

True

In [35]:
movies = pd.read_csv('ml-1m/movies.dat', delimiter='::', engine='python', header=None)
movies.columns = 'MovieID::Title::Genres'.split('::')
movies.head()

print "The following movie id's are missing from movie table"
print sorted(set(range(1, 3953)) - set(movies.MovieID))
print 

mi = ratings['MovieID'].unique()
mi.sort()
#print sorted(set(range(1, 3953)) - set(mi)) 
# movie id have some missing values in addition to the missing values above
# , i.e., there are movies that are not rated by any user.

movie_index_to_ID = dict(zip(range(movies.MovieID.unique().shape[0]), movies.MovieID))
#movie_ID_to_index = dict(zip(movies.MovieID, range(movies.MovieID.unique().shape[0])))
movie_ID_to_index = {k: v for v, k in movie_index_to_ID.iteritems()}

print movie_index_to_ID.items()[-10:]
print movie_ID_to_index.items()[-10:]


The following movie id's are missing from movie table
[91, 221, 323, 622, 646, 677, 686, 689, 740, 817, 883, 995, 1048, 1072, 1074, 1182, 1195, 1229, 1239, 1338, 1402, 1403, 1418, 1435, 1451, 1452, 1469, 1478, 1481, 1491, 1492, 1505, 1506, 1512, 1521, 1530, 1536, 1540, 1560, 1576, 1607, 1618, 1634, 1637, 1638, 1691, 1700, 1712, 1736, 1737, 1745, 1751, 1761, 1763, 1766, 1775, 1778, 1786, 1790, 1800, 1802, 1803, 1808, 1813, 1818, 1823, 1828, 1838, 3815]

[(3873, 3943), (3874, 3944), (3875, 3945), (3876, 3946), (3877, 3947), (3878, 3948), (3879, 3949), (3880, 3950), (3881, 3951), (3882, 3952)]
[(3943, 3873), (3944, 3874), (3945, 3875), (3946, 3876), (3947, 3877), (3948, 3878), (3949, 3879), (3950, 3880), (3951, 3881), (3952, 3882)]


In [54]:
from scipy.sparse import csr_matrix
row = ratings['MovieID'].values
col = ratings['UserID'].values
data = ratings['Rating'].values
ratings_matrix = csr_matrix((data, (row, col)))



In [55]:
ratings_matrix.shape

(3949, 11)

In [16]:
def trace_dot(X, Y):
    """Trace of np.dot(X, Y.T)."""
    return np.dot(X.ravel(), Y.ravel())
norm_X = np.dot(X.data, X.data)
norm_WH = trace_dot(np.dot(np.dot(W.T, W), H), H)
cross_prod = trace_dot((X * H.T), W)
error = np.sqrt(norm_X + norm_WH - 2. * cross_prod)

In [17]:
error/X.shape[0]/X.shape[1]

0.00012841350131919519

In [13]:
from pyspark import SparkConf, SparkContext
sc = spark.sparkContext

In [14]:

sc.setLogLevel("WARN")

def parse_data(line):
    fields = line.split("::")
    tup =  (int(fields[0]), int(fields[1]), float(fields[2]), int(fields[3][-1]))
    return tup


In [10]:
from pyspark.sql.functions import rand

def kfold_cv_generator(dataset, n_folds=3, seed = None):
    if not seed:
        seed = 0
    h = 1.0 / n_folds
    randCol = "cv_rand"
    df = dataset.select("*", rand(seed).alias(randCol)).cache()
    for i in range(n_folds):
        validateLB = i * h
        validateUB = (i + 1) * h
        condition = (df[randCol] >= validateLB) & (df[randCol] < validateUB)
        validation = df.filter(condition)
        train = df.filter(~condition)
        yield train.drop(randCol), validation.drop(randCol)

In [25]:
ratings_sp = spark.createDataFrame(ratings)

In [26]:
ratings.columnsumns

Index([u'UserID', u'MovieID', u'Rating', u'Timestamp'], dtype='object')

In [27]:
ratings_sp.columns

['UserID', 'MovieID', 'Rating', 'Timestamp']

In [29]:
import numpy as np
import os
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

def parse_data(line):
    fields = line.split("::")
    tup =  (int(fields[0]), int(fields[1]), float(fields[2]), int(fields[3][-1]))
    return tup

u_data_file = os.path.join("ml-1m", "ratings.dat")
ratings = sc.textFile(u_data_file).map(parse_data).cache()#.filter(lambda x: x is not None)

#ratings_train = ratings.filter(lambda x: x[3] < 8).map(lambda x: x[:-1])
#ratings_test = ratings.filter(lambda x: x[3] >= 8).map(lambda x: x[:-1])

#print("number of train points: %d" % ratings_train.count())
#print("number of test points: %d" % ratings_test.count())

n_folds = 3
MSE = np.zeros(n_folds)
for i, (ratings_train, ratings_test) in enumerate(kfold_cv_generator(ratings_sp, n_folds)):
    
    print ratings_train.count()
    # TODO: Remove timestamp column from the begining
    model = ALS.train(ratings_train.select(['UserID', 'MovieID', 'Rating']), rank = 10, iterations = 10)

    test_data = ratings_test.rdd.map(lambda r: (r[0], r[1]))
    predictions = model.predictAll(test_data).map(lambda r: ((r[0], r[1]), r[2]))
    p = predictions.collect()
    ratesAndPreds = ratings_test.rdd.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
    MSE[i] = (ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean())**.5

    print("Mean Squared Error = " + str(MSE))


666731
Mean Squared Error = [ 0.91369634  0.          0.        ]
666515
Mean Squared Error = [ 0.91369634  0.92062809  0.        ]
667172
Mean Squared Error = [ 0.91369634  0.92062809  0.91434021]


In [102]:
np.mean([i[1] for i in p])

1.1469104811548383

In [54]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder



lr = LogisticRegression(maxIter=10)

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, np.logspace(-2, 2, 5)) \
    .build()

cnt = 0

class Evaluator:
    def evaluate(self, dataset):
        global cnt
        cnt += 1
        print dataset.count()
        print cnt
        #print dataset['features'].collect()
        #print dataset['label']
        print
        return 1
    
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          #evaluator=BinaryClassificationEvaluator(),
                          evaluator=Evaluator(),
                          numFolds=3)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training)



33
1

33
2

33
3

33
4

33
5

28
6

28
7

28
8

28
9

28
10

39
11

39
12

39
13

39
14

39
15



AttributeError: Evaluator instance has no attribute 'isLargerBetter'

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


In [4]:
import os
spark_home = os.environ['SPARK_HOME']
training = spark.read.format("libsvm").load(os.path.join(spark_home, "data/mllib/sample_libsvm_data.txt"))


In [52]:
training

DataFrame[label: double, features: vector]